In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
def test_eq(a,b): assert a==b, f'{a}, {b}'

In [ ]:
uri = ('https://message-from-space.readthedocs.io/en/latest/_downloads' + 
      '/822b494d13eb35a97f159e041a004fa9/radio-transmission-recording.wav'
)
target = 'message_1.wav'
!wget {uri} -O {target}

--2020-07-05 15:47:39--  https://message-from-space.readthedocs.io/en/latest/_downloads/822b494d13eb35a97f159e041a004fa9/radio-transmission-recording.wav
Resolving message-from-space.readthedocs.io (message-from-space.readthedocs.io)... 104.17.33.82, 104.17.32.82
Connecting to message-from-space.readthedocs.io (message-from-space.readthedocs.io)|104.17.33.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5450806 (5.2M) [audio/x-wav]
Saving to: ‘message_1.wav’

message_1.wav       100%[===================>]   5.20M  8.49MB/s    in 0.6s    

2020-07-05 15:47:40 (8.49 MB/s) - ‘message_1.wav’ saved [5450806/5450806]



In [ ]:
!